In [1]:
%load_ext autoreload
%autoreload

import io
import os
import sys
import django
import glob
import json
import re
import requests
from IPython.core.debugger import set_trace
from pathlib import Path
from django.core.files.base import ContentFile
from rest_framework import serializers
from tqdm.notebook import tqdm
from google.cloud import vision
from google.cloud import storage
from PIL import Image, ImageSequence
from gensim.models import Doc2Vec
from IPython.core.debugger import set_trace
import IPython
import collections
from googletrans import Translator
from google.cloud import translate_v2 as translate

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)
django.setup()

import getch.models as m
from django.db.models import F, Q, Sum, Count, Case, When, IntegerField, Value
from django.apps import apps
from django_currentuser.middleware import get_current_user
from datetime import datetime, timedelta
import time

# 한글태그 필드 채우기

In [10]:
get_ko = lambda x: ' '.join(re.compile(r'[\u3131-\u3163\uac00-\ud7a3]+|[ㄱ-ㅣ]+').findall(x))

for _pix in tqdm(m.Pix.objects.order_by('-id')[:]):
    _tokens = _pix.tokens
    _pix.tokens_ko = get_ko(_tokens)
    _pix.save()

  0%|          | 0/4298 [00:00<?, ?it/s]

# 선택한(pos) & 선택하지않은(neg) 토큰 쿼리하기

In [2]:
# tokens = m.BalancegameRecord.objects.filter(who__user__email='sohee991110@gmail.com').select_related('pix_0', 'pix_1', 'pix_0__tokens_ko', 'pix_1__tokens_ko').annotate(
tokens = m.BalancegameRecord.objects.filter(who__user__email='sohee991110@gmail.com').annotate(    
    pos_tokens=Case(
        When(chosen=0, then=F('pix_0__tokens_ko')),
        When(chosen=1, then=F('pix_1__tokens_ko')),
        default=Value(''),
    ),
    neg_tokens=Case(
        When(chosen=0, then=F('pix_1__tokens_ko')),
        When(chosen=1, then=F('pix_0__tokens_ko')),
        default=Value(''),
    ),
)

In [3]:
pos_tokens = tokens.values_list('pos_tokens', flat=True)
neg_tokens = tokens.values_list('neg_tokens', flat=True)

In [4]:
pos_stat = collections.Counter(' '.join(pos_tokens).split(' ')); pos_stat
neg_stat = collections.Counter(' '.join(neg_tokens).split(' ')); neg_stat;

In [5]:
(pos_stat - neg_stat).most_common(10)

[('자켓', 17),
 ('티셔츠', 14),
 ('휴양', 13),
 ('몸짓', 12),
 ('가방', 11),
 ('청', 11),
 ('야구모자', 10),
 ('반바지', 9),
 ('복부', 9),
 ('캡', 9)]

In [6]:
st = time.time()
collections.Counter(' '.join(pos_tokens).split());
time.time() - st

0.0006799697875976562

In [7]:
st = time.time()
collections.Counter(' '.join(neg_tokens).split());
time.time() - st

0.0006458759307861328